<a href="https://colab.research.google.com/github/coco-maru/samurai_final_assignment/blob/main/%E8%85%B8%E6%B4%BB%E3%82%B9%E3%82%B3%E3%82%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jaconv

  Preparing metadata (setup.py) ... done
  Created wheel for jaconv: filename=jaconv-0.4.0-py3-none-any.whl size=18228 sha256=93b77486a50bc2d0e18b686a054d527fd8ca7f511b302bcbabbb1b2e4c84f87a
  Stored in directory: /root/.cache/pip/wheels/c9/97/53/1f827ebf916b899520ce663227497206525c432be72cc29265
Successfully built jaconv


In [2]:
# ライブラリのインポート
import pandas as pd
from google.colab import files
import io
import jaconv
import re

In [3]:
# 料理データセットの読み込み
uploaded = files.upload()  # ファイル選択ダイアログを出す
file_name = list(uploaded.keys())[0]
xls = pd.ExcelFile(io.BytesIO(uploaded[file_name]))  # Excelファイルのシート名を確認
print(xls.sheet_names)  # Excelのシート名の確認

df_recipes = pd.read_excel(xls, sheet_name='recipes')
df_prebiotics = pd.read_excel(xls, sheet_name='prebiotics')
df_probiotics = pd.read_excel(xls, sheet_name='probiotics')

Saving 料理名と食材・栄養素のデータセット.xlsx to 料理名と食材・栄養素のデータセット.xlsx
['recipes', 'prebiotics', 'probiotics']


In [10]:
# 列をすべて表示
pd.set_option('display.max_columns', None)

# 行をすべて表示（必要に応じて）
pd.set_option('display.max_rows', None)
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム]
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし・精白米（水稲）,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米・精白米（水稲）,60.0,大麦・押麦,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米・精白米（水稲）,70.0,えのきたけ＿生,7.0,しめじ・ぶなしめじ＿生,7.0,まいたけ＿生,7.0,油揚げ,5.0,かつお・昆布だし,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒・上撰,7.5,糸みつば・葉＿生,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米・精白米（水稲）,66.0,油揚げ,5.0,にんじん・根、皮つき＿生,7.0,板こんにゃく（精粉こんにゃく）,15.0,ごぼう・根＿生,20.0,生しいたけ＿生,5.0,若鶏・むね、皮なし＿生,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒・上撰,1.2,さやいんげん・若ざや＿生,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯（もち米製品）,150.0,ごま＿いり,1.0,食塩,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_prebiotics.head()

,野菜類,豆類,フルーツ,海藻類,きのこ類,その他
0,アスパラガス,大豆,バナナ,わかめ,しいたけ,はちみつ
1,ブロッコリー,レンズ豆,りんご,昆布,えのきたけ,NaN
2,キャベツ,ひよこ豆,ベリー,ひじき,まいたけ,NaN
3,玉ねぎ,インゲン豆,キウイ,のり,えのき,NaN
4,ごぼう,NaN,NaN,NaN,NaN,NaN


In [22]:
# データの前処理（料理データセットを正規化表現にする）
# 不要な情報の削除
def clean_foodname(name):
  """「・」「＿」以降の単語を削除する"""
  name = re.sub("・.*", "", name)
  name = re.sub("＿.*", "", name)
  return name

df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)


/tmp/ipython-input-3657101725.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)


TypeError: expected string or bytes-like object, got 'float'

In [19]:
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム]
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米,60.0,大麦・押麦,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米,70.0,えのきたけ＿生,7.0,しめじ・ぶなしめじ＿生,7.0,まいたけ＿生,7.0,油揚げ,5.0,かつお・昆布だし,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒・上撰,7.5,糸みつば・葉＿生,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米,66.0,油揚げ,5.0,にんじん・根、皮つき＿生,7.0,板こんにゃく（精粉こんにゃく）,15.0,ごぼう・根＿生,20.0,生しいたけ＿生,5.0,若鶏・むね、皮なし＿生,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒・上撰,1.2,さやいんげん・若ざや＿生,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯（もち米製品）,150.0,ごま＿いり,1.0,食塩,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
